# Snow Pit Data Ingest
---
## Raw Data Ingest

In [3]:
import xmltodict
import pandas as pd
pd.options.display.max_columns = None

# create dict from xml
# file_path = 'pit_obs.xml'
file_path = '../raw_data/sp/all-MT.xml'
with open(file_path, "rb") as file:
    pit_data = xmltodict.parse(file)

# create dataframe from dict
pit_obs = pd.DataFrame.from_dict(pit_data['Pit_Data']['Pit_Observation'])
pit_obs.columns = [col.replace('@', '') for col in pit_obs.columns]
print(pit_obs.shape)
pit_obs.sample(3)

(500, 52)


,activities,iLayerNumber,iDepth,bld,nid,stability,aviPit,crownObs,aviLoc,incline,winDir,skiAreaPit,bcPit,testPit,windspeed,aspect,sky,windLoading,heightOfSnowpack,precip,serial,version,pitNotes,timestamp,measureFrom,depthUnits,coordType,elvUnits,longType,latType,longitude,lat,zone,north,east,rhoUnits,fractureCat,hardnessScaling,state,range,share,User,Location,Layer,Shear_Test_Result,Density_Profile,Temperature_Profile,bootPen,skiPen,surfGrainType,surfGrainSize,tempUnits
317,Ski tracks on slope; We skied slope;,,,7.77,37155,Good,false,false,,23,,false,,false,,70,BKN,,79,NO,drupal-nid-37155-1641340687000,7.77-0.1,,1641326400000,bottom,cm,UTM,ft,W,N,,,12T,5047156,503522,kg_cubic_m,shear_quality,linear,MT,Gallatin Range-N,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'UTM', '@zone': '12T', '@...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","{'@code': 'ECT', '@sdepth': '58.0', '@depthUni...","{'@profile': '', '@depthUnits': 'cm', '@densit...",NaN,50,20,NaN,NaN,C
350,Ski tracks on slope;,,,7.77,36654,,false,false,,39,N,false,,false,Light Breeze,104,X,no,74,S2,drupal-nid-36654-1640650274000,7.77-0.1,"Shane, Ella, and Erika",1640541900000,bottom,cm,lat_long,ft,W,N,-110.9304220,45.8097200,12T,5072909,505406,kg_cubic_m,shear_quality,linear,MT,Bridger Range,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '59.0', '@depthUn...","{'@profile': '', '@depthUnits': 'cm', '@densit...",NaN,40,NaN,NaN,NaN,C
9,"Collapsing, widespread; Cracking; Instability ...",6,29,7.91,45238,Poor,false,false,,29,S,false,,false,Light Breeze,135,OVC,no,89,S2,drupal-nid-45238-1669954610000,7.91-0.1,Very poor structure below a very unconsolidate...,1669923900000,bottom,cm,lat_long,ft,W,N,-113.491644,48.255926,12U,5347747,315062,kg_cubic_m,fracture_character,linear,MT,Southern Glacier National Park,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '12U'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'PST', '@sdepth': '29.0', '@score':...",{'@profile': '89.0:120;79.0:109;68.0:105;58.0:...,{'@temp_profile': '89.0:-5.5;80.0:-6.5;70.0:-6...,NaN,NaN,MFcl,1,NaN


---
## Flatten Columns

User, Location and Density_Profile can be flattened into columns and concat'd to df

In [4]:
cols_to_flatten = ['User', 'Location', 'Density_Profile']
pit_obs_2 = pit_obs.copy()

for ctf in cols_to_flatten:
    df = pd.DataFrame(pit_obs[ctf].tolist())
    df.columns = [col.replace('@', f'{ctf}_') for col in df.columns]
    pit_obs_2 = pd.concat([pit_obs_2, df], axis='columns')

pit_obs_2.drop(columns=cols_to_flatten, inplace=True)
pit_obs_2.set_index('nid', inplace=True)
pit_obs_2.sample(3)


,activities,iLayerNumber,iDepth,bld,stability,aviPit,crownObs,aviLoc,incline,winDir,skiAreaPit,bcPit,testPit,windspeed,aspect,sky,windLoading,heightOfSnowpack,precip,serial,version,pitNotes,timestamp,measureFrom,depthUnits,coordType,elvUnits,longType,latType,longitude,lat,zone,north,east,rhoUnits,fractureCat,hardnessScaling,state,range,share,Layer,Shear_Test_Result,Temperature_Profile,bootPen,skiPen,surfGrainType,surfGrainSize,tempUnits,User_measureFrom,User_depthUnits,User_tempUnits,User_coordType,User_elvUnits,User_longType,User_latType,User_longitude,User_lat,User_zone,User_north,User_east,User_rhoUnits,User_fractureCat,User_hardnessScaling,User_state,User_range,User_share,User_username,User_useSymbols,User_first,User_last,User_name,User_phone,User_prof,User_email,User_affil,Location_ID,Location_type,Location_zone,Location_ns,Location_ew,Location_east,Location_north,Location_lat,Location_longitude,Location_elv,Location_state,Location_range,Location_name,Density_Profile_profile,Density_Profile_densityUnits,Density_Profile_depthUnits
nid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38658,Pit dug in a Ski Area;,,,7.77,,false,false,,21,SE,true,false,false,Light Breeze,0,OVC,yes,96,S-1,drupal-nid-38658-1643084364000,7.77-0.1,Pit dug in rocky area. Lower snowpack (especia...,1643053800000,top,cm,UTM,m,W,N,,,12T,5006065,466580,kg_cubic_m,fracture_character,exponential,MT,Madison Range-N,true,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","{'@code': 'ECT', '@sdepth': '', '@depthUnits':...",NaN,15,NaN,MFpc,0.5,C,top,cm,C,UTM,m,W,N,,,12T,5006065,466580,kg_cubic_m,fracture_character,exponential,MT,Madison Range-N,true,nata.deleeuw,true,Nata,de Leeuw,Nata de Leeuw,403-627-4891,true,nata.deleeuw@gmail.com,Yellowstone Club Ski Patrol,,UTM,12T,N,W,466580,5006065,,,2830,MT,Madison Range-N,Eglise-2022-01-24,,kg_cubic_m,cm
45310,Ski tracks on slope; We skied slope;,2,64,7.91,Good,false,false,,,SW,false,,false,Light Breeze,90,CLR,previous,75,NO,drupal-nid-45310-1670108769000,7.91-0.1,,1670097600000,bottom,cm,lat_long,ft,W,N,-111.0128870,45.4818490,12T,5036480,498993,kg_cubic_m,shear_quality,linear,MT,Gallatin Range-N,true,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '20.0', '@score':...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN,bottom,cm,F,lat_long,ft,W,N,-111.0128870,45.4818490,12T,5036480,498993,kg_cubic_m,shear_quality,linear,MT,Gallatin Range-N,true,wnhubbard,true,Wyatt,Hubbard,Wyatt Hubbard,8475252738,false,wyatt_hubbard@aol.com,Unaffiliated / Individual,,LATLONG,12T,N,W,498993,5036480,45.4818490,-111.0128870,8315,MT,Gallatin Range-N,H Rock Upper Meadow,,kg_cubic_m,cm
41019,"Collapsing, localized; We skied slope;",3,178,7.77,Poor,false,false,,29,SW,false,,false,Light Breeze,195,SCT,previous,209,NO,drupal-nid-41019-1646085980000,7.77-0.1,Triggered a collapse and whumph while breaking...,1646004600000,bottom,cm,lat_long,ft,W,N,-109.9451100,45.0717500,,,,kg_cubic_m,shear_quality,linear,MT,Cooke City,true,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '179.0', '@depthU...",NaN,NaN,NaN,NaN,NaN,C,bottom,cm,C,lat_long,ft,W,N,-109.9451100,45.0717500,,,,kg_cubic_m,shear_quality,linear,MT,Cooke City,true,amarienthal,true,Alex,Marienthal,Alex Marienthal,3038274788,true,alexmarienthal1@gmail.com,Gallatin National Forest Avalanche Center,,LATLONG,,N,W,,,45.0717500,-109.9451100,10169,MT,Cooke City,"Rasta Chutes, Scotch B",,kg_cubic_m,cm


In [5]:
pit_obs_2.User_username.value_counts()

amarienthal      58
ianhoyer         41
davidzinn        34
hahamarshall     30
nata.deleeuw     29
                 ..
Nelsonvt          1
sallyrose         1
PaulHeffernan     1
Seberia8          1
aschoening        1
Name: User_username, Length: 93, dtype: int64

---
## Split Layer and Shear_Test_Result into separate dataframes

In [8]:
layers = pd.DataFrame()

for pit_index, layer_list in pit_obs_2.Layer.items():
    if isinstance(layer_list, dict): # in case pit has only 1 layer, which will be a dict instead of a list
        layer_list = [layer_list]
    if isinstance(layer_list, list):
      df = pd.DataFrame(layer_list)
      df['nid'] = pit_index
      layers = pd.concat([layers, df], axis='rows')

layers.columns = [col.replace('@', '') for col in layers.columns]
layers.head(10)

,grainSizeUnits1,grainSizeUnits2,grainSize,grainSize1,grainType,grainType1,hardness1,hardness2,layerNumber,waterContent,startDepth,endDepth,multipleHardness,multipleGrainType,multipleGrainSize,nid
0,mm,mm,,,,,F,,1,D,110,100,false,false,false,45338
1,mm,mm,,,,,1F,,2,D,100,90,false,false,false,45338
2,mm,mm,,,,,4F-,,3,D,90,65,false,false,false,45338
3,mm,mm,,,,,4F,,4,D,65,35,false,false,false,45338
4,mm,mm,,,,,4F+,4F+,5,D,35,0,false,false,false,45338
0,mm,mm,,,,,F-,,1,,75,74.5,false,false,false,45310
1,mm,mm,,,,,F,,2,,74.5,64,false,false,false,45310
2,mm,mm,,,,,4F,4F+,3,,64,40,false,false,false,45310
3,mm,mm,,,,,1F,P-,4,,40,20,false,false,false,45310
4,mm,mm,,,,,4F-,,5,,20,0,false,false,false,45310


In [7]:
shear_test_results = pd.DataFrame()

for pit_index, shear_test_result_list in pit_obs_2.Shear_Test_Result.items():
    if isinstance(shear_test_result_list, dict): # in case pit has only 1 shear test, which will be a dict instead of a list
        shear_test_result_list = [shear_test_result_list]
    if isinstance(shear_test_result_list, list):
        df = pd.DataFrame(shear_test_result_list)
        df['nid'] = pit_index
        shear_test_results = pd.concat([shear_test_results, df], axis='rows')

shear_test_results.columns = [col.replace('@', '') for col in shear_test_results.columns]
shear_test_results.sample(3)

# are character and fractureCharacter the same?

,code,sdepth,score,ecScore,ctScore,quality,dateString,numberOfTaps,fractureCat,fractureCharacter,s,lengthOfCut,lengthOfColumn,releaseType,dataCode,nid,depthUnits
0,ECT,,ECTX,,,,02/24/2022,,shear_quality,,ECTX 02/24/2022.12:00:00:0,,,,,40805,cm
0,ECT,212.0,ECTN,12,,,01/19/2022,12,shear_quality,,ECTN12 212.0 01/19/2022.13:15:00:0,,,,,38308,cm
0,ECT,15.0,ECTN,18,,,12/12/2021,18,fracture_character,,ECTN18 15.0 12/12/2021.13:30:00:0,,,,,35629,cm
